In [3]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/lakme/c/604?"
params = "ptype=brand&id=604&root=brand_menu,top_brands,Lakme"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:

        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Lakme",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/lakme/c/604?ptype=brand&id=604&root=brand_menu,top_brands,Lakme
Scraping Page 2: https://www.nykaa.com/brands/lakme/c/604?page_no=2&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/lakme/c/604?page_no=3&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/lakme/c/604?page_no=4&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/lakme/c/604?page_no=5&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping Page 6: https://www.nykaa.com/brands/lakme/c/604?page_no=6&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping Page 7: https://www.nykaa.com/brands/lakme/c/604?page_no=7&sort=popularity&ptype=brand&id=604&root=brand_menu,top_brands,Lakme&eq=desktop
Scraping

In [5]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Lakme', 'Product Name': 'Lakme Rouge Bloom Matte Lip Liquid - Blushing Bae 204(3.5ml)', 'Product Rating': '4.8/5', 'Product Rating & Review Count': '202ratings&156reviews', 'Product Original Price': '₹1199', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Lakme', 'Product Name': 'Lakme Xtraordin-Airy Mattereal Mousse Foundation, Matte Finish, With SPF8, 01 Classic Ivory(25g)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '51630ratings&4709reviews', 'Product Original Price': '₹999', 'Product Offer Price': '₹599', 'Product Discount': '40% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Lakme', 'Product Name': 'Lakme Sun Expert Dry Matte Fluid SPF 50++++ Sunscreen with 1% Niac & Ceramide, for Oily Skin(50ml)', 'Product Rating': '4.9/5', 'Product Rating & Review Count': '430ratings&410reviews', 

In [6]:
lakme = pd.DataFrame(products)
lakme

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Lakme,Lakme Rouge Bloom Matte Lip Liquid - Blushing ...,4.8/5,202ratings&156reviews,₹1199,N/A,No Discount,None,In Stock
1,Lakme,Lakme Xtraordin-Airy Mattereal Mousse Foundati...,4.4/5,51630ratings&4709reviews,₹999,₹599,40% Off,BESTSELLER,In Stock
2,Lakme,Lakme Sun Expert Dry Matte Fluid SPF 50++++ Su...,4.9/5,430ratings&410reviews,₹499,₹449,10% Off,None,In Stock
3,Lakme,"Lakme 9 To 5 Powerplay Priming Foundation, Bui...",4.4/5,47870ratings&3163reviews,₹699,₹384,45% Off,None,In Stock
4,Lakme,"Lakme Unreal Undercover Gel Face Primer, Enric...",4.4/5,75912ratings&5068reviews,₹899,₹539,40% Off,None,In Stock
...,...,...,...,...,...,...,...,...,...
323,Lakme,Lakme Flawless Base Combo(3 pcs),4.4/5,76807ratings&6789reviews,₹2673,₹1722,36% Off,None,Out Of Stock
324,Lakme,Lakme's Haute Nails Selection - Set Of 4(24ml),4.2/5,118ratings&3reviews,₹760,N/A,No Discount,None,Out Of Stock
325,Lakme,Lakme Micellar Pure Facewash + Makeup Remover ...,4.4/5,1146ratings&157reviews,₹749,₹487,35% Off,None,Out Of Stock
326,Lakme,Lakme Prep Cover Set Multipurpose Combo(3 pcs),4.4/5,101089ratings&7148reviews,₹2024,₹1316,35% Off,None,Out Of Stock


In [7]:
# lakme.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Lakme Products_Nykaa.csv", index = False)